1. what is the approximate depth of a Decision Tree trained(without restrictions) on a traing set with 1 million instance?

2. Is a node's Gini impurity generally lower or greater than its parent's? Is it generaaly lover/greater, or always lower/greater?

3. If a Decision Treee is overfitting the training set, is it a good idea to try decreasing max_depth?

If a Decision tree is overfitting the training set, it may be a good idea to decrease max_depth, since this will constrain the model, regularizing it.

4. If a Decision Tree is overfitting the training set, is it a good idea to try scaling the input features?


Decision tree donot care whether or not the training set is scaled or centered; that's one of the nice things about them. so if a decision tree underfits the training set, scaling the input features will just be a waste of time.

5. If it takes one hour to train a Decision tree on a training set containing 1 million instance, roughly how much time will it take to train another Decision tree on a training set containing 10 million instances?

6. If you training set contains 100,000 instances, will setting presort = True speed up training?

7. Train and fine-tune a Decision Tree for the moon dataset.

In [1]:
from sklearn.datasets import make_moons
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
X,y = make_moons(n_samples=10000, noise=0.4, random_state=42)

In [3]:
X.shape

(10000, 2)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [6]:
params = {'max_leaf_nodes':list(range(2,100)), 'min_samples_split':[2,3,4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)
grid_search_cv.fit(X_train,y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    3.3s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'min_samples_split': [2, 3, 4], 'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, v

In [7]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=17,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [8]:
grid_search_cv.best_score_

0.8555

In [9]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

In [10]:
y_pred2 = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred2)

0.8695

8. Grow a forest

a. Continuing the previous exercise, generate 1,000 subset of the training set, each containing 100 instance selected randomly.

In [11]:
from sklearn.model_selection import ShuffleSplit

In [23]:
len(X_train)

8000

In [12]:
n_trees =1000
n_instance = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train)-n_instance, random_state=42)

In [13]:
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b. Train one Decision tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these Decision tree will likely perform worse than the first Decision tree, achieving only about 80% accuracy.

In [14]:
from sklearn.base import clone

In [17]:
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
np.mean(accuracy_scores)

0.8054494999999999

c. Now comes the magic, For each test set instance, generate the predictions of the 1,000 Decision Tress, and keep only the most frequent prediction. This gives you majority -vote predictions over the best set.

In [20]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [21]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model(about 0.5 higher)

In [22]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872